In [23]:
'''讀取每行txt檔案，並將每行文字以逗號分隔，並放入data[]陣列，所需取得的資料：時間在GPGGA，經緯度在GPFLL，衛星編號與強度在GPGSV，
因此透過data陣列的第一個值進行判斷進入想取值的那行。由於要取得台灣時間，但GPGGA的值是UTC時間，故須將時間+8小時；
經緯度的部分則是依照公式將GPGLL的值轉換成google map可讀之格式，緯度公式為前兩個值+上後面值/60，經度公式為前三個值+上後面值/60；
而衛星編號與訊號強度則是在GPGSV，但因為*之前的格式有5種，為了不讓數值被順序影響所以先將GPGSV的行以*分隔，並透過一個變數cn，
計算每行當中包含多少衛星，接著再透過GPGSV的第一個數字與第二個數字判斷現在在第幾行，以輸出正確的編號與強度，並且可以透過第三個數字，
確認衛星數量是否正確，最後將每個部份所獲得的值都放在lines[]陣列裡面，並打開一個可寫的txt文件將輸出都寫入。
其中有透過folium地圖視覺化工具，先設定map畫面位置與zoom in大小，再把抓到的經緯度標記到map上，最後把標記完的map輸出到html，完成加分題。'''

import folium #地圖視覺化工具
myMap = folium.Map([24.970097, 121.26733516666667], zoom_start=50) #設定map畫面位置與zoom in大小
f = open('GPS_Data.txt', 'r')     #讀檔
path = 'output.txt'               #寫檔用
f1 = open(path, 'w')
lines= []                         #用來放每個部份所獲得的值
for line in f.readlines():        #以每行方式進行讀取
    data= []                      
    #print(line)
    data2=[]
    data3=[]
    for num in line.strip().split(','):               #以逗號分割
        data.append(num)                              #將分割完字串放入data陣列
    if data[0]=='$GPGGA':                             #從GPGGA取得時間
        count = 0               
        times=int(data[1][0:2])+8                     #GPGGA的值是UTC時間，故須將時間+8小時
        time = ['台灣時區時間: ',str(times),':',data[1][2:4],':',data[1][4:6],'\n']         #打時間輸出格式
        lines.append(time)
        f1.writelines(time)                                                                #寫入output.txt
    elif data[0]=='$GPGLL':                             #從GPGLL取得經緯度
        if data[1] != '' and data[3] != '':
            map1= float(data[1][0:2])+float(data[1][2:])/60  #緯度公式為前兩個值+上後面值/60
            #print(float(data[1][0:2]))
            #print(float(data[1][2:]))
            #print(map1)
            map2= float(data[3][0:3])+float(data[3][3:])/60  #經度公式為前三個值+上後面值/60
            #print(float(data[3][0:3]))
            #print(float(data[3][3:]))
            #print(map2)
            
            myMap.add_child(folium.Marker(location=[map1,map2],popup=''))              #把抓到的經緯度標記到map上
            loc = ['緯度: ',str(map1),data[2],' 經度: ',str(map2),data[4],'\n\n']       #打經緯度輸出格式
            lines.append(loc)
            f1.writelines(loc)                                         #寫入output.txt
    elif data[0]=='$GPGSV':                                 #從GPSV取得衛星編號、訊號強度
        for gsv in line.strip().split('*'):                 #為了不讓數值被順序影響所以先將GPGSV的行以*分隔
            data2.append(gsv)                               #將分割完字串放入data2陣列
        #print(data2[0])
        c=0
        for num2 in data2[0].strip().split(','):            #以逗號分割
            c+=1
            data3.append(num2)                              #將分割完字串放入data3陣列
        #print(data3[0])
        #print(c)
        c1=int(data3[1])                                    #GPGSV的第一個數值，此回總共有幾行GPGSV，放在c1
        c2=int(data3[2])                                    #GPGSV的第二個數值，此行為這回的第幾行GPGSV，放在c2
        cn=c/4-1                                            #每行GPGSV最多只有4個衛星，故透過公式計算這行要幾個衛星
        #print(cn)
              
        if data3[0]=='$GPGSV' and c2<c1:                                   #若此GPGSV不是最後一行則進入判斷
            sv = ['衛星編號: ',data3[4],' ','訊號強度: ',data3[7],'\n',
                  '衛星編號: ',data3[8],' ','訊號強度: ',data3[11],'\n',
                  '衛星編號: ',data3[12],' ','訊號強度: ',data3[15],'\n',
                  '衛星編號: ',data3[16],' ','訊號強度: ',data3[19],'\n']   #輸出每顆衛星的編號與強度
            lines.append(sv)
            f1.writelines(sv)                                              #寫入output.txt
        if data3[0]=='$GPGSV' and c2==c1 and cn==4:                        #若此GPGSV是最後一行而衛星數有4個
            sv1 = ['衛星編號: ',data3[4],' ','訊號強度: ',data3[7],'\n',
                  '衛星編號: ',data3[8],' ','訊號強度: ',data3[11],'\n',
                  '衛星編號: ',data3[12],' ','訊號強度: ',data3[15],'\n',
                  '衛星編號: ',data3[16],' ','訊號強度: ',data3[19],'\n']   #輸出每顆衛星的編號與強度
            lines.append(sv1)
            f1.writelines(sv1)                                             #寫入output.txt

        if data3[0]=='$GPGSV' and c2==c1 and cn==3:                        #若此GPGSV是最後一行而衛星數有3個
            sv1 = ['衛星編號: ',data3[4],' ','訊號強度: ',data3[7],'\n',
                  '衛星編號: ',data3[8],' ','訊號強度: ',data3[11],'\n',
                  '衛星編號: ',data3[12],' ','訊號強度: ',data3[15],'\n',]   #輸出每顆衛星的編號與強度
            lines.append(sv1)
            f1.writelines(sv1)                                             #寫入output.txt

        if data3[0]=='$GPGSV' and c2==c1 and cn==2:                        #若此GPGSV是最後一行而衛星數有2個
            sv1 = ['衛星編號: ',data3[4],' ','訊號強度: ',data3[7],'\n',
                  '衛星編號: ',data3[8],' ','訊號強度: ',data3[11],'\n',]   #輸出每顆衛星的編號與強度
            lines.append(sv1)
            f1.writelines(sv1)                                             #寫入output.txt

        if data3[0]=='$GPGSV' and c2==c1 and cn==1:                        #若此GPGSV是最後一行而衛星數有1個
            sv1 = ['衛星編號: ',data3[4],' ','訊號強度: ',data3[7],'\n',]   #輸出每顆衛星的編號與強度
            lines.append(sv1)
            f1.writelines(sv1)                                             #寫入output.txt
myMap.save('myMap.html')                                                   #把標記完的map輸出到html
f1.close()                                                                 #關閉寫入檔
f.close                                                                    #關閉讀取檔

<function TextIOWrapper.close()>